In [73]:
from queue import PriorityQueue
from bitarray import bitarray
from collections import defaultdict

In [124]:
def static_codes(text):
    counts = defaultdict(lambda: 0)
    codes = defaultdict(lambda: "")

    # Liczę wystąpienia poszczególnych liter w tekście
    for letter in text:
        counts[letter] += 1

    # Wrzucam wszystko do kolejki w postaci:
    # (ilość, indeks (żeby klucze były unikatowe), {litera})
    Q = PriorityQueue()
    for e in map(lambda x: (x[1][1], x[0], {x[1][0]}), enumerate(counts.items())):
        Q.put(e)

    while not Q.empty():
        a = Q.get()

        if Q.empty():
            break

        b = Q.get()

        # Odświeżam kody wszystkich potomków obecnego węzła w drzewie
        for child in a[2]:
            codes[child] = "0" + codes[child]
        for child in b[2]:
            codes[child] = "1" + codes[child]

        # Wstawiam do kolejki wszystkich potomków jako nowy węzeł
        children = a[2] | b[2]
        Q.put((a[0] + b[0], a[1], children))

    return codes


def static_encode(text, codes):
    compressed = bitarray()

    for letter in text:
        compressed.extend(codes[letter])

    return compressed


def static_compress(text, output):
    codes = static_codes(text)
    compressed = static_encode(text, codes)

    with open(output, "w") as file:
        file.write(str(len(codes)))
        file.write(str(len(compressed)))

        for letter, code in codes.items():
            file.write(letter)
            file.write(str(len(code)))
    


In [103]:
text = "asdfasdasa"

In [125]:
static_compress(text, "file")

In [ ]:
def dynamic_encode(text):
    pass


In [ ]:
def decode(path):
    with open(path, "rb") as file:
        pass

In [ ]:
def compress(alghorithm):
    pass
